In [ ]:
# https://www.youtube.com/watch?v=DqtlR0y0suo

In [4]:
#Sending GET request through Insomnia API to nfl/odds page. Data does not contain props. Will have to run this script 2x
import requests
import pandas as pd


get_content_url= "https://content.unabated.com/markets/game-odds/v_gameodds.json?"
get_books_url = "https://api-k.unabated.com/api/users/settings"


#Main Lines URL
Privilege_content = "https://content.unabated.com/markets/game-odds/b_gameodds.json"
#This one is when I am logged in

In [5]:
url = Privilege_content
#returns isPrivileged: true for premium sportsbooks

payload = ""
headers = {
    "authority": "content.unabated.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9",
    "dnt": "1",
    "origin": "https://unabated.com",
    "referer": "https://unabated.com/",
    "sec-ch-ua": "^\^Google",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "^\^Windows^^",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "x-amz-cf-v-id": "dNSob7rQOiJFE_59NDSm7tuZ6ed5MA_8w8CHVqr8tGSFYpgzEUqBuQ=="
}
#We can customize the headers we include ^ I left as it came
r = requests.request("GET", url, data=payload, headers=headers)

# print(r.json())
# all good

data = r.json()

marketSources = (data["marketSources"])
teams = (data["teams"])
gameOddsEvents = (data["gameOddsEvents"])


df_sportsbooks = pd.DataFrame(marketSources)
df_teams = pd.DataFrame(teams)
df_teams=df_teams.transpose()


Transforming the SportsBooks Reference Table

In [7]:
#Removing columns for building purposes 
#Note a bunch of other sportsbooks exist but are not shown on page. 

# df_sb = pd.DataFrame().assign(Name=df_sportsbooks['name'],bookId=df_sportsbooks['id'],isActive=df_sportsbooks['isActive'],
# isPrivileged=df_sportsbooks['isPrivileged'],statusId=df_sportsbooks['statusId'],modifiedDate=df_sportsbooks['modifiedOn'])
df_sb = pd.DataFrame().assign(Name=df_sportsbooks['name'],bookId=df_sportsbooks['id'],isActive=df_sportsbooks['isActive'],
statusId=df_sportsbooks['statusId'],modifiedDate=df_sportsbooks['modifiedOn'])

#Creating Binary flags 
 
df_sb['nyBook']= 0
df_sb['sharpBook']= 0


In [8]:
df_nyBooks= pd.DataFrame(
    {"Name" : ["Caesars", "DraftKings", "FanDuel", "FanDuel - Delayed", "BetMGM", "PointsBet", "BetRivers", "WynnBet", "Resorts", "BallyBet"],
    'nyBook': [1,1,1,1,1,1,1,1,1,1]
    }
)
df_sharpBooks = pd.DataFrame(
    {"Name" :['Unabated','Pinnacle','Pinnacle - Delayed','Bookmaker','Prophet Exchange'],
    'sharpBook': [1,1,1,1,1]
    }
)

In [9]:
df_sb=df_sb.set_index('Name')
df2 = df_nyBooks.set_index('Name')
df_sb.update(df2)

df3=df_sharpBooks.set_index('Name')
df_sb.update(df3)

In [10]:
df_sb

#This is the Sportsbook Reference table. Might also want to change the number format for the dates

,bookId,isActive,statusId,modifiedDate,nyBook,sharpBook
Name,,,,,,
Circa - Delayed,68,True,2,2022-12-21T19:37:56.422057,0.0,0.0
WynnBet,33,True,2,2022-10-31T16:22:07.064598,1.0,0.0
Sugarhouse,27,True,3,2022-12-18T20:14:17.691736,0.0,0.0
Sporttrade,67,True,1,2022-12-17T22:15:38.503501,0.0,0.0
Caesars,20,True,2,2022-11-08T15:00:33.647225,1.0,0.0
...,...,...,...,...,...,...
Bet365,14,True,3,2021-07-07T02:13:52.730139,0.0,0.0
Retail Consensus,50,False,2,2022-04-07T22:20:45.343532,0.0,0.0
YouWager,56,True,2,2022-12-30T22:02:10.81532,0.0,0.0


In [16]:
#Limit my calculations to nybooks only

nybooks_df = df_sb.loc[df_sb['nyBook'] == 1]
nySharpbooks_df = df_sb.loc[(df_sb['nyBook'] == 1) | (df_sb['sharpBook'] == 1)]

In [17]:
nySharpbooks_df


,bookId,isActive,statusId,modifiedDate,nyBook,sharpBook
Name,,,,,,
WynnBet,33,True,2,2022-10-31T16:22:07.064598,1.0,0.0
Caesars,20,True,2,2022-11-08T15:00:33.647225,1.0,0.0
Unabated,49,True,0,2022-03-16T21:38:37.525262,0.0,1.0
Pinnacle - Delayed,58,True,2,2022-08-17T12:18:20.442277,0.0,1.0
FanDuel - Delayed,57,True,2,2022-06-09T14:45:56.826297,1.0,0.0
DraftKings,1,True,1,2022-06-28T15:42:41.593641,1.0,0.0
BetMGM,4,True,3,2022-07-28T13:36:35.331082,1.0,0.0
PointsBet,3,True,2,2022-08-27T03:32:48.307845,1.0,0.0
Prophet Exchange,66,True,1,2022-11-16T22:12:44.347255,0.0,1.0


In [34]:
df_sb.to_csv('sportsbooksTable.csv')

________________________________________________________________________________________


Now we create the Teams table. 

The eventId is the unique gameID which two teams will share. Not positive on what sideId is yet, but it distinguishes one team from another inside of a cell. 


In [18]:
#slimming down

df_teams_refined = pd.DataFrame().assign(teamId=df_teams['id'],teamName=df_teams['name'],abbreviation=df_teams['abbreviation'],
eventId=df_teams['eventId'],sideId=df_teams['sideId'],leagueId=df_teams['leagueId'],divisionId=df_teams['divisionId'],
modifiedDate=df_teams['modifiedOn'])

df_teams_refined
#This is our Teams Table

,teamId,teamName,abbreviation,eventId,sideId,leagueId,divisionId,modifiedDate
2,2,Atlanta Falcons,ATL,21959,1,1,8,0001-01-01T00:00:00
30,30,Tampa Bay Buccaneers,TB,21959,0,1,8,0001-01-01T00:00:00
4,4,Buffalo Bills,BUF,21960,1,1,1,0001-01-01T00:00:00
19,19,New England Patriots,NE,21960,0,1,1,0001-01-01T00:00:00
6,6,Chicago Bears,CHI,21961,1,1,7,0001-01-01T00:00:00
...,...,...,...,...,...,...,...,...
535,535,NC Asheville,UNCA,39047,0,4,None,0001-01-01T00:00:00
1175,1175,North Dakota State,NDS,40030,0,2,10,0001-01-01T00:00:00
1073,1073,South Dakota State,SDS,40030,1,2,10,0001-01-01T00:00:00
689,689,TCU,TCU,40492,0,2,9,0001-01-01T00:00:00


In [33]:
df_teams_refined.to_csv('df_teams_ref_out.csv')



__________________________________________________________________________________

Now we tackle the GameOdd Table

In [19]:
data = []
for i in gameOddsEvents:
  for j in gameOddsEvents[i]:
    event_teams = j['eventTeams']
    id_0 = event_teams['0']['id']
    id_1 = event_teams['1']['id']
    
    # Extract the americanPrice values from gameOddsMarketSourcesLines
    game_odds_market_sources_lines = j['gameOddsMarketSourcesLines']
    for market_source, market_data in game_odds_market_sources_lines.items():
      siX= market_source
      for bet_type, bet_data in market_data.items():
            american_price = bet_data['americanPrice']
            marketLineId= bet_data['marketLineId']
            marketId= bet_data['marketId']
            marketSourceId= bet_data['marketSourceId']
            points= bet_data['points']
            price= bet_data['price']
            sourcePrice= bet_data['sourcePrice']
            btX = bet_type
            # Store the extracted data in a tuple
            row = (i, j['eventId'],j['eventStart'],j['eventEnd'] ,id_0, id_1, siX, btX, marketLineId, marketSourceId,
            marketId, points,american_price, price, sourcePrice)
            data.append(row)



# Create the DataFrame with the extracted data
df_MainLineOdds = pd.DataFrame(data, columns=['lgX', 'eventId','eventStart','eventEnd','Teamid_0', 'Teamid_1', 'siX', 'btX', 'marketLineId','marketSourceId','marketId','points','american_price','price','sourcePrice'])
df_MainLineOdds['lgX_league']= df_MainLineOdds['lgX'].str.get(2)
df_MainLineOdds['lgX_pt']= df_MainLineOdds['lgX'].str.get(6)
df_MainLineOdds['lgX_stage']= df_MainLineOdds['lgX'].str[8:]
df_MainLineOdds['siX_side']= df_MainLineOdds['siX'].str.get(2)

df_MainLineOdds

,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,marketLineId,marketSourceId,marketId,points,american_price,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side
0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,67201420,1,474203,-4.0,-110,-110,1.909091,1,1,pregame,1
1,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,67201450,1,667433,0.0,-190,-190,1.526316,1,1,pregame,1
2,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,249616595,1,1578122,40.5,-110,-110,1.909091,1,1,pregame,1
3,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,270812471,2,474203,-4.5,-110,-110,-110.000000,1,1,pregame,1
4,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt1,270812473,2,667433,0.0,-210,-210,-210.000000,1,1,pregame,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41767,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt3,271684190,1,1615466,30.5,-115,-115,1.869565,2,3,pregame,1
41768,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt1,271684225,1,1615468,0.0,-285,-285,1.350877,2,3,pregame,1
41769,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt2,271684169,1,1615465,6.0,-110,-110,1.909091,2,3,pregame,0
41770,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt3,271684209,1,1615467,30.5,-105,-105,1.952381,2,3,pregame,0


In [20]:
bet_type = {
    'bt1': 'Money_Line',
    'bt2':'Spread',
    'bt3': 'Total_Combined'}

#There is a bt4 : 'Combined' I have not seen it exisst in the data

In [34]:
print(bet_type['bt1'])

Money_Line


In [32]:
df_MainLineOdds.to_csv('MainLineOdds.csv')

Write to MainLinesMaster

In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\ammoo\OneDrive\Desktop\PythonOutData\Main Lines Master.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_MainLineOdds.to_excel(writer, sheet_name='MainLineOdds')
df_teams_refined.to_excel(writer, sheet_name='Teams')
df_sb.to_excel(writer, sheet_name='Books')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

c:\Users\ammoo\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Create Table where only bets for nySharpbooks_df

In [118]:
filtered_df = df_MainLineOdds[df_MainLineOdds['marketSourceId'].isin(nySharpbooks_df['bookId'])].copy()

In [119]:
filtered_df

,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,marketLineId,marketSourceId,marketId,points,american_price,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side
0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,67201420,1,474203,-4.0,-110,-110,1.909091,1,1,pregame,1
1,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,67201450,1,667433,0.0,-190,-190,1.526316,1,1,pregame,1
2,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,249616595,1,1578122,40.5,-110,-110,1.909091,1,1,pregame,1
3,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,270812471,2,474203,-4.5,-110,-110,-110.000000,1,1,pregame,1
4,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt1,270812473,2,667433,0.0,-210,-210,-210.000000,1,1,pregame,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41767,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt3,271684190,1,1615466,30.5,-115,-115,1.869565,2,3,pregame,1
41768,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt1,271684225,1,1615468,0.0,-285,-285,1.350877,2,3,pregame,1
41769,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt2,271684169,1,1615465,6.0,-110,-110,1.909091,2,3,pregame,0
41770,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt3,271684209,1,1615467,30.5,-105,-105,1.952381,2,3,pregame,0


Insert the Bet Type from the bet type dictionary and TeamName_1, sideId_team1, TeamName_0, sideId_team0 from Teams table. We identify which team the bet is placed for by comparing the bet's siX_side to the appropriate sideId_team0/1 

In [120]:
# Insert the bet_type

# filtered_df['bet_type'] = filtered_df['btX'].map(bet_type)
filtered_df.loc[:, 'bet_type'] = filtered_df['btX'].map(bet_type)

In [126]:
#Insert the TeamNames and side(home/away) for Teamid_0 and Teamid_1
# Merge the two dataframes on teamId
filtered_df = df_teams_refined.merge(filtered_df, left_on='teamId', right_on='Teamid_0')

# Rename the columns so they don't overwrite each other
filtered_df = filtered_df.rename(columns={'teamName':'TeamName_0', 'abbreviation':'TeamAbbrev_0','sideId':'sideId_team0'})

# Drop the now unnecessary merged columns column
filtered_df = filtered_df.drop(columns={'teamId','TeamAbbrev_0','eventId_x','leagueId','divisionId','modifiedDate'})


#repeat for Teamid_1
filtered_df = filtered_df.merge(df_teams_refined, left_on='Teamid_1', right_on='teamId')
filtered_df = filtered_df.rename(columns={'teamName':'TeamName_1', 'abbreviation':'TeamAbbrev_1','sideId':'sideId_team1'})
filtered_df = filtered_df.drop(columns=['teamId','TeamAbbrev_1','leagueId','divisionId','modifiedDate','eventId'])
filtered_df = filtered_df.rename(columns={'eventId_y' : 'eventId'})

In [123]:
filtered_df

,TeamName_0,sideId_team0,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,...,american_price,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side,bet_type,TeamName_1,sideId_team1
0,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,...,-110,-110,1.909091,1,1,pregame,1,Spread,Atlanta Falcons,1
1,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,...,-190,-190,1.526316,1,1,pregame,1,Money_Line,Atlanta Falcons,1
2,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,...,-110,-110,1.909091,1,1,pregame,1,Total_Combined,Atlanta Falcons,1
3,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,...,-110,-110,-110.000000,1,1,pregame,1,Spread,Atlanta Falcons,1
4,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt1,...,-210,-210,-210.000000,1,1,pregame,1,Money_Line,Atlanta Falcons,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16697,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt3,...,-115,-115,1.869565,2,3,pregame,1,Total_Combined,Georgia,1
16698,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt1,...,-285,-285,1.350877,2,3,pregame,1,Money_Line,Georgia,1
16699,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt2,...,-110,-110,1.909091,2,3,pregame,0,Spread,Georgia,1
16700,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt3,...,-105,-105,1.952381,2,3,pregame,0,Total_Combined,Georgia,1


In [122]:
filtered_df.columns

Index(['TeamName_0', 'sideId_team0', 'lgX', 'eventId', 'eventStart',
       'eventEnd', 'Teamid_0', 'Teamid_1', 'siX', 'btX', 'marketLineId',
       'marketSourceId', 'marketId', 'points', 'american_price', 'price',
       'sourcePrice', 'lgX_league', 'lgX_pt', 'lgX_stage', 'siX_side',
       'bet_type', 'TeamName_1', 'sideId_team1'],
      dtype='object')

Calc ImpliedProb%

In [61]:
# I am going to create a column called Implied Probability. To calculate implied probability, 
# we are going to look to the american_price column. 

# These are the rules: 
# btX = bt1
# If the odds are positive, do 100 / (odds + 100)
# So for +400, it would be 100 / (400 + 100) = 100/500 = 20% Implied WP
# If the odds are negative, do odds / (odds + 100)
# So for -300, it would be 300 / (300 + 100) = 300/400 = 75% Implied WP -->


# def calculate_implied_probability(row):
#     if row['american_price'] > 0:
#         return 100 / (row['american_price'] + 100)
#     else:
#         return row['american_price'] / (row['american_price'] + 100)

# filtered_df['ImpliedProb%'] = filtered_df.apply(calculate_implied_probability, axis=1)



#We assume that implied probability is calculated the same for all bet types

# def calc_implied_probability_MoneyLine(row):
#     if row['btX'] == 'bt1':
#         if row['american_price'] > 0:
#             return 100 / (row['american_price'] + 100)
#         elif row['american_price'] < 0:
#             return row['american_price'] / (row['american_price'] + 100)
#     return None

# filtered_df['ImpliedProb%'] = filtered_df.apply(calc_implied_probability_MoneyLine, axis=1)


In [124]:
def implied_probability(row):

        if row['american_price'] > 0:
            return 1 / (row['american_price']/100 + 1) *100
        elif row['american_price'] < 0:
            return 1/(1+100/abs(row['american_price'])) *100
    

filtered_df['ImpliedProb%'] = filtered_df.apply(implied_probability, axis=1)

In [125]:
filtered_df

,TeamName_0,sideId_team0,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,...,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side,bet_type,TeamName_1,sideId_team1,ImpliedProb%
0,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,...,-110,1.909091,1,1,pregame,1,Spread,Atlanta Falcons,1,52.380952
1,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,...,-190,1.526316,1,1,pregame,1,Money_Line,Atlanta Falcons,1,65.517241
2,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,...,-110,1.909091,1,1,pregame,1,Total_Combined,Atlanta Falcons,1,52.380952
3,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,...,-110,-110.000000,1,1,pregame,1,Spread,Atlanta Falcons,1,52.380952
4,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt1,...,-210,-210.000000,1,1,pregame,1,Money_Line,Atlanta Falcons,1,67.741935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16697,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt3,...,-115,1.869565,2,3,pregame,1,Total_Combined,Georgia,1,53.488372
16698,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt1,...,-285,1.350877,2,3,pregame,1,Money_Line,Georgia,1,74.025974
16699,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt2,...,-110,1.909091,2,3,pregame,0,Spread,Georgia,1,52.380952
16700,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt3,...,-105,1.952381,2,3,pregame,0,Total_Combined,Georgia,1,51.219512
